# 네이버 금융
- selenium
- iframe
    - HTML 안에 또 다른 HTML

In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
import pandas as pd
import time
from selenium import webdriver

In [21]:
url ="https://finance.naver.com/marketindex/"
driver = webdriver.Chrome("../driver/chromedriver")
driver.get(url)

In [5]:
# 환전 고시 환율
contents = driver.find_elements_by_css_selector("#exchangeList > li > a") #은 id, 
contents[0].text

'1,183.50\n3.00'

In [6]:
contents[0].find_element_by_css_selector(".value").text #elements 아니고 element, class가 value인것

'1,183.50'

In [7]:
#주요 뉴스 제목
news = driver.find_elements_by_css_selector("#content > div.section_news > div > ul > li:nth-child(1) > p > a")
news[0].text

'“원자재가 상승세 둔화…상대적으로 원유·구'

In [75]:
# 밑 페이지에 있는 매매기준율
driver.find_elements_by_css_selector("body > div > table > tbody > tr")
# iframe은 접근하면 빈 리스트가 나오게 된다. 
# 위에 html 안에 또다른 html 영역이라 바로 접근 힘들다.

[]

# iframe

In [76]:
# selenium을 이용한 방법

In [12]:
# iframe 태그 지정
iframe = driver.find_element_by_css_selector("#frame_ex1")
# 같은 방법
# driver.find_element_by_id("frame_ex1")

In [13]:
# iframe 태그로 이동
driver.switch_to_frame(iframe) #경고문구 무시

C:\Users\whfh1\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.frame instead
  


In [14]:
# 환전 고시 환율
contents = driver.find_elements_by_css_selector("#exchangeList > li > a")
contents[0].find_element_by_css_selector(".value").text

IndexError: list index out of range

In [107]:
# 에러가 나는 것이 당연하다, iframe으로 태그를 옮겼기 때문에 앞에서 실행되었던 코드들은 실행되지 않는다.

In [15]:
driver.find_element_by_css_selector('.sale').text

'1,183.50'

In [16]:
#가져오려는 코드 위의 부모코드인 .sale을 먼저 불렀기 때문에 다음의 코드는 실행된다
contents = driver.find_elements_by_css_selector("body > div > table > tbody > tr")
contents[0].text

'미국 USD 1,183.50 1,204.21 1,162.79 1,195.00 1,172.00 1.000'

In [17]:
contents[0].find_element_by_css_selector(".sale").text #위의 실행 코드 중 맨 앞에것만 가져와진다.

'1,183.50'

In [19]:
datas = []

for content in contents: #미국 유럽 일본 세개만,,
    title = content.find_element_by_css_selector(".tit>a").text
    sale = content.find_element_by_css_selector(".sale").text
    link = content.find_element_by_css_selector(".tit>a").get_attribute("href")
    datas.append({
        "title": title,
        "sale" : sale,
        "link" : link
    })
    
df = pd.DataFrame(datas)
df.to_excel("./sel_naver_finance.xlsx", encoding = "utf-8")
df


,title,sale,link
0,미국 USD,"1,183.50",https://finance.naver.com/marketindex/exchange...
1,유럽연합 EUR,"1,373.10",https://finance.naver.com/marketindex/exchange...
2,일본 JPY (100엔),"1,036.20",https://finance.naver.com/marketindex/exchange...


In [20]:
driver.quit()

In [22]:
# request를 이용한 방법 - 요청할 url을 알고 있어야 한다.

In [23]:
import requests
from bs4 import BeautifulSoup

In [24]:
url = "https://finance.naver.com/marketindex/exchangeList.naver"
response = requests.get(url)
response

<Response [200]>

In [25]:
response.content

b'<html lang=\'ko\'>\n<head>\n<title>\xb3\xd7\xc0\xcc\xb9\xf6 \xb1\xdd\xc0\xb6</title>\n<meta http-equiv=\'Content-Type\' content=\'text/html; charset=euc-kr\'>\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/finance.css">\n\n<script language="javascript">document.domain="naver.com";</script>\n<script type="text/javascript" src="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/js/info/jindo.min.ns.1.5.3.euckr.js"></script>\n<script type="text/javascript" src="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/js/lcslog.js"></script>\n</head>\n<body>\n<div class="tbl_area">\n\t<table border="1" class="tbl_exchange" summary="\xc8\xaf\xc0\xfc \xb0\xed\xbd\xc3 \xc8\xaf\xc0\xb2 \xb8\xae\xbd\xba\xc6\xae">\n\t<caption>\xc8\xaf\xc0\xfc \xb0\xed\xbd\xc3 \xc8\xaf\xc0\xb2</caption>\n\t<colgroup>\n\

In [27]:
soup = BeautifulSoup(response.content, "html.parser")
print(soup.prettify())

<html lang="ko">
 <head>
  <title>
   네이버 금융
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="text/javascript" http-equiv="Content-Script-Type"/>
  <meta content="text/css" http-equiv="Content-Style-Type"/>
  <link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/finance.css" rel="stylesheet" type="text/css"/>
  <script language="javascript">
   document.domain="naver.com";
  </script>
  <script src="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript">
  </script>
  <script src="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/js/lcslog.js" type="text/javascript">
  </script>
 </head>
 <body>
  <div class="tbl_area">
   <table border="1" class="tbl_exchange" summary="환전 고시 환율 리스트">
    <caption>
     환전 고시 환율
    </caption>
    <colgroup>
     <col width="162"/>
     <col width="92"/>
     <col width="92"/>
     <col width="92"/>
     <col widt

In [28]:
soup.select_one(".sale").text

'1,183.50'

In [30]:
contents = soup.select("tbody > tr")
contents[2].select_one(".sale").text

'1,036.20'

In [31]:
for content in contents:
    print((content.select_one(".sale").text))

1,183.50
1,373.10
1,036.20
183.97
152.19
42.32
1,627.49
3,074.11
956.90
1,282.51
137.44
878.39
836.14
54.10
1.44
127.69
0.42
367.32
184.54
140.40
315.53
3,921.86
3,139.84
322.21
1,669.25
75.29
35.42
877.87
284.63
8.43
324.98
2.78
877.87
15.77
6.91
13.83
23.33
58.20
216.70
5.20
81.03
16.67
3.81
300.39
